In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# Setup splinter
executable_path = {'executable_path': '/Users/annasours/chromedriver/chromedriver'}

# executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# from splinter import Browser
# import pymongo

In [3]:
# URLs of pages to be scraped

# NASA Mars News
url_1 = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
# JPL Mars Space Images - Featured Image
url_2 = "https://www.jpl.nasa.gov/images?search=&category=Mars"
# Mars Facts
url_3 = "https://space-facts.com/mars/"
# Mars Hemispheres
url_4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

### Nasa Mars News

In [ ]:
# Optional delay for loading the page --> accounts for slow internet connection or pictures that load.
#browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

#browser.is_element_present_by_text("Feedback", wait_time=1)

#browser.is_element_present_by_id("email", wait_time=1)

# browser.is_element_present_by_name("email", wait_time=1)

In [4]:
# Try again using broswer
# browser = init_browser()
browser.visit(url_1)

# one way to get the webpage to load slower
time.sleep(2)

In [20]:
html=browser.html
news_soup=BeautifulSoup(html,'html.parser')

In [21]:
first_article=news_soup.find('ul', class_='item_list')

In [37]:
news_title=first_article.find('div', class_='content_title').a.text
news_p=first_article.a.text

print(news_title)
print('-----'*5)
print(news_p)

NASA's Perseverance Mars Rover Mission Honors Navajo Language
-------------------------
Working with the Navajo Nation, the rover team has named features on Mars with words from the Navajo language.NASA's Perseverance Mars Rover Mission Honors Navajo Language


### JPL Mars Space Image - Featured Image

In [38]:
browser.visit(url_2)

# one way to get the webpage to load slower
time.sleep(2)

In [39]:
html=browser.html
jpl_soup=BeautifulSoup(html,'html.parser')

In [54]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image 
# and assign the url string to a variable called featured_image_url.

navbar=jpl_soup.find('div', class_='NavDesktopDropdownContent')\
        .find('img')

navbar

<img alt="" class="BaseImage can-hover:group-hover:scale-100 transition duration-300 ease-in-out transform scale-105 object-cover" data-src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.2e16d0ba.fill-400x225-c70.jpg" height="225" loading="lazy" src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.2e16d0ba.fill-400x225-c70.jpg" width="400"/>

In [56]:
featured_image_url=navbar['data-src']
featured_image_url

'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.2e16d0ba.fill-400x225-c70.jpg'

In [49]:
browser.click_link_by_href(featured_href)

ElementDoesNotExist: no elements could be found with link by href "/images/supercams-mars-meteorite-aboard-the-iss"